In [2]:
# 使用BandIndex优选特征进行实验

In [3]:
# SVM

In [5]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import numpy as np




In [6]:

data = pd.read_excel('D:\datasss\BandIndex.xlsx')


features = data.iloc[:, 3:].values
labels = data.iloc[:, 2].values


scaler = StandardScaler()
featuress = scaler.fit_transform(features)


In [7]:

svm_model = SVC()


kernel = ['linear', 'rbf']


param_grid = {
    'C': [0.1, 1, 10, 100,1000],
    'kernel': kernel,
    'gamma': ['scale', 'auto',1, 3, 5, 10, 100] if 'rbf' in kernel else [None]
}


In [8]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [9]:

random_search = RandomizedSearchCV(svm_model, param_distributions=param_grid, cv=cv, n_iter=70, scoring='f1_weighted',n_jobs=-1)
random_search.fit(featuress, labels)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=SVC(), n_iter=70, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': ['scale', 'auto', 1, 3, 5, 10,
                                                  100],
                                        'kernel': ['linear', 'rbf']},
                   scoring='f1_weighted')

In [10]:

best_params = random_search.best_params_
print("最佳参数：", best_params)

best_score = random_search.best_score_
print("最佳模型的f1_weighted值：", best_score)


最佳参数： {'kernel': 'linear', 'gamma': 'scale', 'C': 10}
最佳模型的f1_weighted值： 0.932286396853281


In [11]:

best_model = random_search.best_estimator_


In [12]:

accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
confusion = np.zeros((4, 4))  


In [13]:
for train_index, test_index in cv.split(featuress, labels):
    X_train, X_test = featuress[train_index], featuress[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)


    print("混淆矩阵：")
    print(cm)
    

    print(u'算法评价:')
    print(classification_report(y_test, y_pred))

    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='macro'))
    recall_scores.append(recall_score(y_test, y_pred, average='macro'))
    f1_scores.append(f1_score(y_test, y_pred, average='macro'))
    confusion += confusion_matrix(y_test, y_pred, labels=np.unique(labels))
    print("=========================================================================")
    


混淆矩阵：
[[ 9  0  0  0]
 [ 0 10  0  2]
 [ 0  0 12  0]
 [ 0  1  0  8]]
算法评价:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.91      0.83      0.87        12
           2       1.00      1.00      1.00        12
           3       0.80      0.89      0.84         9

    accuracy                           0.93        42
   macro avg       0.93      0.93      0.93        42
weighted avg       0.93      0.93      0.93        42

混淆矩阵：
[[ 9  0  0  0]
 [ 0 11  0  1]
 [ 1  0 11  0]
 [ 0  0  0  8]]
算法评价:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       1.00      0.92      0.96        12
           2       1.00      0.92      0.96        12
           3       0.89      1.00      0.94         8

    accuracy                           0.95        41
   macro avg       0.95      0.96      0.95        41
weighted avg       0.96      0.95      

In [14]:
print("Accuracy:", sum(accuracy_scores) / len(accuracy_scores))
print("Precision:", sum(precision_scores) / len(precision_scores))
print("Recall:", sum(recall_scores) / len(recall_scores))
print("F1 Score:", sum(f1_scores) / len(f1_scores))
print("Confusion Matrix:")
print(confusion)


Accuracy: 0.9320557491289199
Precision: 0.9369556138306139
Recall: 0.9308226495726496
F1 Score: 0.9299403115411187
Confusion Matrix:
[[41.  0.  1.  0.]
 [ 0. 56.  0.  4.]
 [ 2.  0. 59.  2.]
 [ 0.  3.  2. 36.]]


In [16]:
# KNN
from sklearn.neighbors import KNeighborsClassifier


In [17]:

knn_model = KNeighborsClassifier()


In [18]:

weights = ['uniform', 'distance']

param_grid1 = {
    'n_neighbors': range(1, 11),
    'weights': weights,
    'p': [1, 2, 3, 4, 5, 6] if 'distance'in weights else [None]
    
}


In [19]:

random_search1 = RandomizedSearchCV(knn_model, param_distributions=param_grid1, cv=cv, n_iter=100, scoring='f1_weighted',n_jobs=-1)
random_search1.fit(featuress, labels)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=KNeighborsClassifier(), n_iter=100, n_jobs=-1,
                   param_distributions={'n_neighbors': range(1, 11),
                                        'p': [1, 2, 3, 4, 5, 6],
                                        'weights': ['uniform', 'distance']},
                   scoring='f1_weighted')

In [20]:

best_params1 = random_search1.best_params_
print("最佳参数：", best_params1)

best_score1 = random_search1.best_score_
print("最佳模型的f1_weighted值：", best_score1)


最佳参数： {'weights': 'distance', 'p': 5, 'n_neighbors': 4}
最佳模型的f1_weighted值： 0.9102212334227181


In [21]:

best_model1 = random_search1.best_estimator_


In [22]:

accuracy_scores1 = []
precision_scores1 = []
recall_scores1 = []
f1_scores1 = []
confusion1 = np.zeros((4, 4))  


In [23]:
for train_index, test_index in cv.split(featuress, labels):
    X_train, X_test = featuress[train_index], featuress[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    best_model1.fit(X_train, y_train)
    y_pred = best_model1.predict(X_test)

    cm1 = confusion_matrix(y_test, y_pred)


    print("混淆矩阵：")
    print(cm1)
    

    print(u'算法评价:')
    print(classification_report(y_test, y_pred))

    accuracy_scores1.append(accuracy_score(y_test, y_pred))
    precision_scores1.append(precision_score(y_test, y_pred, average='macro'))
    recall_scores1.append(recall_score(y_test, y_pred, average='macro'))
    f1_scores1.append(f1_score(y_test, y_pred, average='macro'))
    confusion1 += confusion_matrix(y_test, y_pred, labels=np.unique(labels))
    print("=========================================================================")


混淆矩阵：
[[ 9  0  0  0]
 [ 0 11  0  1]
 [ 0  0 12  0]
 [ 0  2  1  6]]
算法评价:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.85      0.92      0.88        12
           2       0.92      1.00      0.96        12
           3       0.86      0.67      0.75         9

    accuracy                           0.90        42
   macro avg       0.91      0.90      0.90        42
weighted avg       0.90      0.90      0.90        42

混淆矩阵：
[[ 8  0  1  0]
 [ 0 12  0  0]
 [ 1  0 11  0]
 [ 0  2  1  5]]
算法评价:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.86      1.00      0.92        12
           2       0.85      0.92      0.88        12
           3       1.00      0.62      0.77         8

    accuracy                           0.88        41
   macro avg       0.90      0.86      0.87        41
weighted avg       0.89      0.88      

In [24]:
print("Accuracy:", sum(accuracy_scores1) / len(accuracy_scores1))
print("Precision:", sum(precision_scores1) / len(precision_scores1))
print("Recall:", sum(recall_scores1) / len(recall_scores1))
print("F1 Score:", sum(f1_scores1) / len(f1_scores1))
print("Confusion Matrix:")
print(confusion1)


Accuracy: 0.9126596980255517
Precision: 0.9157127594627594
Recall: 0.9034188034188034
F1 Score: 0.9057655456459803
Confusion Matrix:
[[40.  0.  2.  0.]
 [ 0. 57.  0.  3.]
 [ 2.  0. 60.  1.]
 [ 0.  6.  4. 31.]]


In [26]:
# RF
from sklearn.ensemble import RandomForestClassifier


In [27]:

rf_model = RandomForestClassifier()


In [28]:

param_grid2 = {
    'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'bootstrap':[True, False]
}


In [29]:

random_search2 = RandomizedSearchCV(rf_model, param_distributions=param_grid2, cv=cv, n_iter=200, scoring='f1_weighted',n_jobs=-1)
random_search2.fit(featuress, labels)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(), n_iter=200, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200, 250,
                                                         300, 350, 400, 450,
                                                         500]},
                   scoring='f1_weighted')

In [30]:

best_params2 = random_search2.best_params_
print("最佳参数：", best_params2)

best_score2 = random_search2.best_score_
print("最佳模型的f1_weighted值：", best_score2)


最佳参数： {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 20, 'criterion': 'entropy', 'bootstrap': False}
最佳模型的f1_weighted值： 0.9010242136506579


In [31]:

best_model2 = random_search2.best_estimator_


In [32]:

accuracy_scores2 = []
precision_scores2 = []
recall_scores2 = []
f1_scores2 = []
confusion2 = np.zeros((4, 4))  


In [33]:
for train_index, test_index in cv.split(featuress, labels):
    X_train, X_test = featuress[train_index], featuress[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    best_model2.fit(X_train, y_train)
    y_pred = best_model2.predict(X_test)

    cm2 = confusion_matrix(y_test, y_pred)


    print("混淆矩阵：")
    print(cm2)
    

    print(u'算法评价:')
    print(classification_report(y_test, y_pred))

    accuracy_scores2.append(accuracy_score(y_test, y_pred))
    precision_scores2.append(precision_score(y_test, y_pred, average='macro'))
    recall_scores2.append(recall_score(y_test, y_pred, average='macro'))
    f1_scores2.append(f1_score(y_test, y_pred, average='macro'))

    confusion2 += confusion_matrix(y_test, y_pred, labels=np.unique(labels))
    print("=========================================================================")


混淆矩阵：
[[ 9  0  0  0]
 [ 0 11  0  1]
 [ 0  0 12  0]
 [ 0  1  0  8]]
算法评价:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.92      0.92      0.92        12
           2       1.00      1.00      1.00        12
           3       0.89      0.89      0.89         9

    accuracy                           0.95        42
   macro avg       0.95      0.95      0.95        42
weighted avg       0.95      0.95      0.95        42

混淆矩阵：
[[ 8  0  1  0]
 [ 0 11  0  1]
 [ 1  0 10  1]
 [ 0  3  1  4]]
算法评价:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89         9
           1       0.79      0.92      0.85        12
           2       0.83      0.83      0.83        12
           3       0.67      0.50      0.57         8

    accuracy                           0.80        41
   macro avg       0.79      0.78      0.78        41
weighted avg       0.80      0.80      

In [34]:
print("Accuracy:", sum(accuracy_scores2) / len(accuracy_scores2))
print("Precision:", sum(precision_scores2) / len(precision_scores2))
print("Recall:", sum(recall_scores2) / len(recall_scores2))
print("F1 Score:", sum(f1_scores2) / len(f1_scores2))
print("Confusion Matrix:")
print(confusion2)


Accuracy: 0.8831591173054587
Precision: 0.8856691919191919
Recall: 0.8786324786324787
F1 Score: 0.8765756538782854
Confusion Matrix:
[[39.  0.  3.  0.]
 [ 0. 57.  0.  3.]
 [ 4.  0. 54.  5.]
 [ 0.  5.  4. 32.]]


In [36]:
# XGBOOST
import xgboost as xgb


In [37]:

xgb_model = xgb.XGBClassifier()


In [38]:

param_grid3 = {
    'learning_rate': [0.01, 0.015, 0.05, 0.1, 0.2],
    'max_depth': range(3, 12, 2),
    'n_estimators': range(400, 800, 100),
    'min_child_weight': range(1, 6),
    'subsample': [0.6, 0.7, 0.8, 0.9, 1],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1],
    'gamma': [0, 0.05, 0.1, 0.3, 0.5],
    'reg_alpha': [0.05, 0.1, 0.5, 1, 2],
    'reg_lambda': [0.05, 0.1, 0.5, 1, 2]
}


In [39]:

random_search3 = RandomizedSearchCV(xgb_model, param_distributions=param_grid3, cv=cv, n_iter=500, scoring='f1_weighted',n_jobs=-1)
random_search3.fit(featuress, labels)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importan...
                   n_iter=500, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1],
                                    

In [40]:

best_params3 = random_search3.best_params_
print("最佳参数：", best_params3)

best_score3 = random_search3.best_score_
print("最佳模型的f1_weighted值：", best_score3)


最佳参数： {'subsample': 0.6, 'reg_lambda': 0.1, 'reg_alpha': 2, 'n_estimators': 400, 'min_child_weight': 2, 'max_depth': 9, 'learning_rate': 0.2, 'gamma': 0.05, 'colsample_bytree': 0.7}
最佳模型的f1_weighted值： 0.8974733010954411


In [41]:

best_model3 = random_search3.best_estimator_


In [42]:

accuracy_scores3 = []
precision_scores3 = []
recall_scores3 = []
f1_scores3 = []
confusion3 = np.zeros((4, 4))  

In [43]:
for train_index, test_index in cv.split(featuress, labels):
    X_train, X_test = featuress[train_index], featuress[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    best_model3.fit(X_train, y_train)
    y_pred = best_model3.predict(X_test)
   
    cm3 = confusion_matrix(y_test, y_pred)


    print("混淆矩阵：")
    print(cm3)
    

    print(u'算法评价:')
    print(classification_report(y_test, y_pred))

    accuracy_scores3.append(accuracy_score(y_test, y_pred))
    precision_scores3.append(precision_score(y_test, y_pred, average='macro'))
    recall_scores3.append(recall_score(y_test, y_pred, average='macro'))
    f1_scores3.append(f1_score(y_test, y_pred, average='macro'))

    confusion3 += confusion_matrix(y_test, y_pred, labels=np.unique(labels))
    print("=========================================================================")


混淆矩阵：
[[ 9  0  0  0]
 [ 0 12  0  0]
 [ 0  0 12  0]
 [ 0  1  0  8]]
算法评价:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.92      1.00      0.96        12
           2       1.00      1.00      1.00        12
           3       1.00      0.89      0.94         9

    accuracy                           0.98        42
   macro avg       0.98      0.97      0.98        42
weighted avg       0.98      0.98      0.98        42

混淆矩阵：
[[ 9  0  0  0]
 [ 0 11  0  1]
 [ 0  0 11  1]
 [ 0  2  1  5]]
算法评价:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.85      0.92      0.88        12
           2       0.92      0.92      0.92        12
           3       0.71      0.62      0.67         8

    accuracy                           0.88        41
   macro avg       0.87      0.86      0.87        41
weighted avg       0.87      0.88      

In [44]:
print("Accuracy:", sum(accuracy_scores3) / len(accuracy_scores3))
print("Precision:", sum(precision_scores3) / len(precision_scores3))
print("Recall:", sum(recall_scores3) / len(recall_scores3))
print("F1 Score:", sum(f1_scores3) / len(f1_scores3))
print("Confusion Matrix:")
print(confusion3)


Accuracy: 0.8976771196283391
Precision: 0.9067299367299368
Recall: 0.8942841880341881
F1 Score: 0.8936207966440165
Confusion Matrix:
[[40.  0.  2.  0.]
 [ 0. 56.  0.  4.]
 [ 3.  0. 56.  4.]
 [ 0.  4.  4. 33.]]
